In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,auc
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.feature_selection import SelectFromModel

In [2]:
#############################
#Load Data

In [3]:
df_1 = pd.read_csv('Claim_Data.csv',dtype={"clm_dates": str,"aft_ind":int,"pd_amt":float})
df_2 = pd.read_csv('Predictor Dataset.csv')
df_3 = pd.read_csv('Subsequent_Loss_Experience.csv')

In [4]:
df_1 = df_1[df_1['pd_amt']>=1]
df_1['clm_dates']= pd.to_datetime(df_1['clm_dates'])
df_1['clm_dates'] = df_1['clm_dates'].dt.year
df_1 = df_1[df_1['clm_dates'] >= 2012]
df_1 = df_1[df_1['clm_dates'] <= 2016]

In [5]:
df_1['hhld_id'].nunique()

2456

In [6]:
claim_count = df_1.groupby(['hhld_id', 'clm_dates', 'aft_ind']).size().reset_index(name = 'claim_count')
claim_count;

In [7]:
def getAtFaultMatrix(clm_dates, aft_ind, claim_count):
    Initial_List = [0]*10
    clm_year_Index = int(clm_dates)-2012
    aft_ind_Index = aft_ind * 5
    Index = clm_year_Index+aft_ind_Index
    Initial_List[Index] = claim_count
    return Initial_List

In [8]:
claim_count['aft_matrix'] = claim_count.apply(lambda x: getAtFaultMatrix(x.clm_dates, x.aft_ind, x.claim_count), 
                                             axis = 1)

In [9]:
claim_count[['naft_2012','naft_2013','naft_2014','naft_2015','naft_2016',\
            'aft_2012','aft_2013','aft_2014','aft_2015','aft_2016']] \
            = pd.DataFrame(claim_count.aft_matrix.tolist(), index= claim_count.index)

In [10]:
claim_count = claim_count.drop(columns = ['clm_dates', 'aft_ind', 'claim_count', 'aft_matrix'])
df_1 = claim_count.groupby('hhld_id').sum()
df_1.head()

,naft_2012,naft_2013,naft_2014,naft_2015,naft_2016,aft_2012,aft_2013,aft_2014,aft_2015,aft_2016
hhld_id,,,,,,,,,,
3,0,0,0,0,0,0,0,0,0,1
4,0,0,0,1,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0
9,0,0,1,0,0,0,0,0,0,0


In [11]:
df_1.shape

(2456, 10)

In [12]:
df_3 = df_3.drop(columns = ['premium', 'loss_amount', 'loss_ratio'])

In [13]:
df = pd.merge(df_2, df_3, on = 'hhld_id')
df = pd.merge(df, df_1, on = 'hhld_id')
df.shape
df.head()

,hhld_id,drvr_cnt,veh_cnt,min_age,hoh_age,max_age,avg_age,min_mon_lic,hoh_mon_lic,max_mon_lic,...,naft_2012,naft_2013,naft_2014,naft_2015,naft_2016,aft_2012,aft_2013,aft_2014,aft_2015,aft_2016
0,4317,1,1,47.370047,47.370047,47.370047,47.370047,333.539213,290.637858,290.637858,...,0,0,0,0,0,0,0,0,0,1
1,767,1,1,75.346640,75.346640,75.346640,75.346640,712.159675,712.159675,712.159675,...,1,0,0,0,0,0,0,0,0,0
2,3248,2,2,43.674743,48.459073,48.459073,46.066908,332.096922,389.508881,389.508881,...,0,0,2,0,0,0,0,1,0,0
3,2797,1,2,39.956548,39.956548,39.956548,39.956548,287.478578,287.478578,287.478578,...,0,1,0,0,0,0,0,0,0,0
4,1301,1,1,16.247208,16.247208,16.247208,16.247208,2.966494,2.966494,2.966494,...,0,0,1,0,0,0,1,0,0,0


In [14]:
df.to_csv('1_Merged_Data.csv',index=False)